Making this notebook to play with using array based observations rather than images. Let's start smaller.

In [1]:
import gymnasium as gym
from IPython.display import Video
import sys
import os

os.environ["SDL_VIDEODRIVER"] = "dummy" # this stops pygame opening it's own window


In [2]:
%load_ext autoreload

In [3]:
import gymnasium as gym
from environments.model_ready_boids import RavenChasingBoids

from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy


In [4]:
%autoreload 2
# Now import the function
import gym_utils
import controllers
import agents
import runners
import environments
from environments.simple_boids import PureBoinds, BoidsWithRavenoid
from gym_utils.vis import create_environment_video

In [5]:
env = BoidsWithRavenoid(100, render_mode="rgb_array")

In [7]:
env.action_space.sample()

0